# Loading

In [ ]:
!pip install -U -q "google-genai"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.7/159.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import copy
import string
import re
import json
import random
import time

from google import genai
from google.genai import types

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
client = genai.Client(api_key="your-api-key-here")

In [ ]:
for model_info in client.models.list():
    print(model_info.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
m

In [ ]:
for model_info in client.tunings.list():
    print(model_info.name)

tunedModels/salarytunedmodel1-o7ojl7z82wbp
tunedModels/salarytunedmodel1-8m0fccs9srwj
tunedModels/salarytunedmodel1-636kmk97vwof
tunedModels/salarytunedmodel2-3h17mzl216ez
tunedModels/salarytunedmodel3-e6mptc3xnfii


# Dataset Prep

In [ ]:
train_df = pd.read_csv('data/work_arrangements_development_set.csv')
test_df = pd.read_csv('data/work_arrangements_test_set.csv')

In [ ]:
# df to store model predictions
train_format_df = pd.DataFrame(columns=["text_input", "output"])

In [ ]:
test_format_df = pd.DataFrame(columns=["text_input", "output"])

In [ ]:
for i in range(len(train_df)):

  desc = str(train_df.iloc[i].job_ad)
  desc += """
Extract work arrangement from this job description, and classify it into one of the following labels: "Remote", "Hybrid", or "OnSite".
Respond in JSON: {"work_arrangement": ""}.
"""

  label = f'{{"work_arrangement": "{train_df.iloc[i].y_true}"}}.'
  train_format_df.loc[i] = [desc, label]

In [ ]:
train_format_df

,text_input,output
0,Job title: CEO\nAbstract: Exciting opportunity...,"{""work_arrangement"": ""Remote""}."
1,Job title: Home-Based Online ESL Teacher (Onli...,"{""work_arrangement"": ""Remote""}."
2,"Job title: Safeguarding, De La Salle\nAbstract...","{""work_arrangement"": ""Hybrid""}."
3,Job title: Delivery Driver\nAbstract: Pickup t...,"{""work_arrangement"": ""OnSite""}."
4,Job title: Store Supervisor\nAbstract: We are ...,"{""work_arrangement"": ""OnSite""}."
...,...,...
94,Job title: Senior Pipeline Technical Director\...,"{""work_arrangement"": ""Hybrid""}."
95,Job title: Customer Support Administrator\nAbs...,"{""work_arrangement"": ""OnSite""}."
96,Job title: Remote Writing Evaluator for AI (As...,"{""work_arrangement"": ""Remote""}."
97,Job title: People & Culture Advisor\nAbstract:...,"{""work_arrangement"": ""Hybrid""}."


In [ ]:
for i in range(len(test_df)):

  desc = str(test_df.iloc[i].job_ad)
  desc += """
Extract work arrangement from this job description, and classify it into one of the following labels: "Remote", "Hybrid", or "OnSite".
Respond in JSON: {"work_arrangement": ""}.
"""

  label = test_df.iloc[i].y_true
  test_format_df.loc[i] = [desc, label]

In [ ]:
test_format_df

,text_input,output
0,Job title: Restaurant Kitchen Hand\nAbstract: ...,OnSite
1,Job title: Chef\nAbstract: The Carlin Group Pt...,OnSite
2,"Job title: Accounting Associate (AU, NZ, US Ex...",Remote
3,Job title: Occupational Therapist\nAbstract: P...,Hybrid
4,Job title: Social Media Manager Paid Ads\nAbst...,Remote
...,...,...
94,Job title: Dietitian - Aged Care - Hervey Bay/...,OnSite
95,Job title: Customer Support Consultant - Retai...,Hybrid
96,Job title: Aluminium windows & doors - saw ope...,OnSite
97,Job title: Landscape Maintenance (Full Time) L...,OnSite


# Tuning

In [ ]:
# create tuning examples
training_dataset=types.TuningDataset(
        examples=[
            types.TuningExample(
                text_input=train_format_df.iloc[i].text_input,
                output=train_format_df.iloc[i].output,
            )
            for i in range(len(train_format_df))
        ],
    )

In [ ]:
# start tuning job
tuning_job = client.tunings.tune(
    base_model='models/gemini-1.5-flash-001-tuning',
    training_dataset=training_dataset,
    # hyperparameters
    config=types.CreateTuningJobConfig(
        epoch_count= 5,
        batch_size=4,
        learning_rate=0.001,
        tuned_model_display_name="workarr_tuned_model_1"
    )
)

<ipython-input-14-5462268b5876>:2: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  tuning_job = client.tunings.tune(


In [ ]:
tuning_job = client.tunings.get(name=tuning_job.name)
print(tuning_job)

name='tunedModels/workarrtunedmodel1-z4y92tt6e4pj' state=<JobState.JOB_STATE_RUNNING: 'JOB_STATE_RUNNING'> create_time=datetime.datetime(2025, 4, 24, 4, 49, 56, 505551, tzinfo=TzInfo(UTC)) start_time=datetime.datetime(2025, 4, 24, 4, 49, 57, 459201, tzinfo=TzInfo(UTC)) end_time=None update_time=datetime.datetime(2025, 4, 24, 4, 49, 56, 505551, tzinfo=TzInfo(UTC)) error=None description=None base_model='models/gemini-1.5-flash-001-tuning' tuned_model=TunedModel(model='tunedModels/workarrtunedmodel1-z4y92tt6e4pj', endpoint='tunedModels/workarrtunedmodel1-z4y92tt6e4pj') supervised_tuning_spec=None tuning_data_stats=None encryption_spec=None partner_model_tuning_spec=None distillation_spec=None experiment=None labels=None pipeline_job=None tuned_model_display_name=None


In [ ]:
tuning_job

TuningJob(name='tunedModels/workarrtunedmodel1-z4y92tt6e4pj', state=<JobState.JOB_STATE_RUNNING: 'JOB_STATE_RUNNING'>, create_time=datetime.datetime(2025, 4, 24, 4, 49, 56, 505551, tzinfo=TzInfo(UTC)), start_time=datetime.datetime(2025, 4, 24, 4, 49, 57, 459201, tzinfo=TzInfo(UTC)), end_time=None, update_time=datetime.datetime(2025, 4, 24, 4, 49, 56, 505551, tzinfo=TzInfo(UTC)), error=None, description=None, base_model='models/gemini-1.5-flash-001-tuning', tuned_model=TunedModel(model='tunedModels/workarrtunedmodel1-z4y92tt6e4pj', endpoint='tunedModels/workarrtunedmodel1-z4y92tt6e4pj'), supervised_tuning_spec=None, tuning_data_stats=None, encryption_spec=None, partner_model_tuning_spec=None, distillation_spec=None, experiment=None, labels=None, pipeline_job=None, tuned_model_display_name=None)

# Testing

In [ ]:
# fetch the tuning job again by model name
tuning_job = client.tunings.get(name='tunedModels/workarrtunedmodel1-z4y92tt6e4pj')
print(tuning_job)

name='tunedModels/workarrtunedmodel1-z4y92tt6e4pj' state=<JobState.JOB_STATE_SUCCEEDED: 'JOB_STATE_SUCCEEDED'> create_time=datetime.datetime(2025, 4, 24, 4, 49, 56, 505551, tzinfo=TzInfo(UTC)) start_time=datetime.datetime(2025, 4, 24, 4, 49, 57, 459201, tzinfo=TzInfo(UTC)) end_time=datetime.datetime(2025, 4, 24, 4, 54, 48, 856933, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 4, 24, 4, 54, 48, 856933, tzinfo=TzInfo(UTC)) error=None description=None base_model='models/gemini-1.5-flash-001-tuning' tuned_model=TunedModel(model='tunedModels/workarrtunedmodel1-z4y92tt6e4pj', endpoint='tunedModels/workarrtunedmodel1-z4y92tt6e4pj') supervised_tuning_spec=None tuning_data_stats=None encryption_spec=None partner_model_tuning_spec=None distillation_spec=None experiment=None labels=None pipeline_job=None tuned_model_display_name=None


## Qualitative

In [ ]:
test_index = 67

response = client.models.generate_content(
      model='tunedModels/workarrtunedmodel1-z4y92tt6e4pj',
      contents=test_format_df.iloc[test_index].text_input,
  )

response.text

'{"work_arrangement": "Hybrid"}.'

## Quantitative

In [ ]:
# create df to capture results
results_df = pd.DataFrame(columns=["y_true", "y_pred"])

In [ ]:
for i in range(len(test_format_df)):
  while True:
    try:
      response = client.models.generate_content(
          model='tunedModels/workarrtunedmodel1-z4y92tt6e4pj',
          contents=test_format_df.iloc[i].text_input,
      )
      results_df.loc[len(results_df)] = [test_format_df.iloc[i].output, response.text]
      # print(i)
      break  # break the retry loop if successful

    except Exception as e:
      print(e)
      retry_delay = 60  # default delay

      print(f"Waiting {retry_delay}s to retry at index {i}")
      time.sleep(retry_delay)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '38s'}]}}
Waiting 60s to retry at index 15
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
429 RESOURCE_EXHAUSTED. {'er

In [ ]:
# export the dataframe to a new csv file
results_df.to_csv('data/workarr_labelled_test_set_gemini_finetune.csv', index=False)

# Metrics

In [ ]:
preds = pd.read_csv('data/workarr_labelled_test_set_gemini_finetune.csv')
test_df = pd.read_csv('data/work_arrangements_test_set.csv')

In [ ]:
# preprocess training and test sets
for i in range(len(preds)):
  try:
    answer_str = preds.iloc[i].y_pred
    answer_str_ = answer_str[answer_str.find('{'):answer_str.find('}') + 1]
    answer_str_ = answer_str_.replace('“', '"')
    answer_str_ = answer_str_.replace('”', '"')
    answer = json.loads(answer_str_)

    label = answer['work_arrangement']

  except json.JSONDecodeError:
    print(f"Failed to parse model output as JSON: {answer_str}")
    label = "ERROR " + answer_str

  preds.loc[i, 'y_pred'] = label.lower()
  test_df.loc[i, 'y_true'] = test_df.iloc[i].y_true.lower()

In [ ]:
test_df['y_pred'] = preds.y_pred.values

test_df

,id,job_ad,y_true,y_pred
0,79746043,Job title: Restaurant Kitchen Hand\nAbstract: ...,onsite,onsite
1,79796468,Job title: Chef\nAbstract: The Carlin Group Pt...,onsite,onsite
2,80346773,"Job title: Accounting Associate (AU, NZ, US Ex...",remote,remote
3,80015193,Job title: Occupational Therapist\nAbstract: P...,hybrid,hybrid
4,80292327,Job title: Social Media Manager Paid Ads\nAbst...,remote,remote
...,...,...,...,...
94,79930630,Job title: Dietitian - Aged Care - Hervey Bay/...,onsite,onsite
95,79847353,Job title: Customer Support Consultant - Retai...,hybrid,hybrid
96,80052768,Job title: Aluminium windows & doors - saw ope...,onsite,onsite
97,80088290,Job title: Landscape Maintenance (Full Time) L...,onsite,onsite


In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(
    test_df.y_true.values, test_df.y_pred.values, labels=['remote', 'hybrid', 'onsite']
)

precision_macro, recall_macro, f1_score_macro, _ = precision_recall_fscore_support(
    test_df.y_true.values, test_df.y_pred.values, average='macro'
)

precision_micro, recall_micro, f1_score_micro, _ = precision_recall_fscore_support(
    test_df.y_true.values, test_df.y_pred.values, average='micro'
)

precision_weighted, recall_weighted, f1_score_weighted, _ = precision_recall_fscore_support(
    test_df.y_true.values, test_df.y_pred.values, average='weighted'
)

acc = accuracy_score(test_df.y_true.values, test_df.y_pred.values)

In [ ]:
print(f'Accuracy: {acc}')
pd.DataFrame(
    {
        'Precision': [*precision, "", precision_macro, precision_weighted, precision_micro],
        'Recall': [*recall, "", recall_macro, recall_weighted, recall_micro],
        'F1-score': [*f1_score, "", f1_score_macro, f1_score_weighted, f1_score_micro],
    },
    index=['Remote', 'Hybrid', 'OnSite', "", 'macro average', 'weighted average', 'micro average']
)

Accuracy: 0.9595959595959596


,Precision,Recall,F1-score
Remote,0.928571,1.0,0.962963
Hybrid,1.0,0.888889,0.941176
OnSite,0.957447,0.978261,0.967742
,,,
macro average,0.962006,0.955717,0.957294
weighted average,0.961469,0.959596,0.959242
micro average,0.959596,0.959596,0.959596


In [ ]:
print(classification_report(test_df.y_true.values, test_df.y_pred.values, labels=['remote', 'hybrid', 'onsite']))

              precision    recall  f1-score   support

      remote       0.93      1.00      0.96        26
      hybrid       1.00      0.89      0.94        27
      onsite       0.96      0.98      0.97        46

    accuracy                           0.96        99
   macro avg       0.96      0.96      0.96        99
weighted avg       0.96      0.96      0.96        99

